# The Very Basics of Musical Instruments Classification using Machine Learning
## MFCC, Deep Learning (Gentle introduction using Keras), Sequential Model, Grid Search

<br>

<p align="left">
<img src="./img/businesscard.jpg" width="300px" alt="Business Card" align="left" >
</p>
<br>



In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/ZlxZ3i0RP9I" frameborder="0" 
allow="accelerometer; encrypted-media; gyroscope; picture-in-picture"></iframe>

## Imports

In [2]:
# Imports

#General
import numpy as np
import pickle
import itertools

# System
import os, fnmatch

# Visualization
import seaborn #visualization library, must be imported before all other plotting libraries
import matplotlib.pyplot as plt
from IPython.core.display import HTML, display

# Machine Learning
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import classification_report

# Deep Learning
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import SGD
from keras.callbacks import History
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

# Random Seed
from tensorflow import set_random_seed
from numpy.random import seed
seed(1)
set_random_seed(2)

# Audio
import librosa.display, librosa

# Configurations
path='./audio/london_phill_dataset_multi/'

Using TensorFlow backend.


## Auxiliary Functions

In [3]:
# Function to Display a Website
def show_web(url):
    html_code='<center><iframe src="%s" width="800" height="600" frameborder="0" marginheight="0" marginwidth="0">Loading...</iframe></center>' \
		% (url)
    display(HTML(html_code))

## Get filenames

In [4]:
# Get files in data path

files = []
for root, dirnames, filenames in os.walk(path):
    for filename in fnmatch.filter(filenames, '*.mp3'):
        files.append(os.path.join(root, filename))

print("found %d audio files in %s"%(len(files),path))

found 600 audio files in ./audio/london_phill_dataset_multi/


## Load Labels

In [5]:
filename="inst_labels.pl"
# Load labels
with open(filename, "rb") as f:
    labels = pickle.load( open( filename, "rb" ) )

In [6]:
# Encode Labels
labelencoder = LabelEncoder()
labelencoder.fit(labels)
print(len(labelencoder.classes_), "classes:", ", ".join(list(labelencoder.classes_)))
classes_num = labelencoder.transform(labels)

6 classes: cello, flute, oboe, sax, trumpet, viola


## Parameters for MFCC

In [7]:
# Parameters
# Signal Processing Parameters
fs = 44100         # Sampling Frequency
n_fft = 2048       # length of the FFT window
hop_length = 512   # Number of samples between successive frames
n_mels = 128       # Number of Mel bands
n_mfcc = 13        # Number of MFCCs

# Machine Learning Parameters
testset_size = 0.25 #Percentage of data for Testing
n_neighbors=1       # Number of neighbors for kNN Classifier

## Save / Load Feature Vector

In [8]:
filename="mfcc_feature_vectors.pl"
# Load mfcc features from saved file
with open(filename, "rb") as f:
    scaled_feature_vectors = pickle.load( open( filename, "rb" ) )

## Train and Test Sets

In [9]:
# Create Train and Test Set
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0)
splits = splitter.split(scaled_feature_vectors, classes_num)
for train_index, test_index in splits:
    train_set = scaled_feature_vectors[train_index]
    test_set = scaled_feature_vectors[test_index]
    train_classes = classes_num[train_index]
    test_classes = classes_num[test_index]

In [10]:
# Check Set Shapes
print("train_set shape:",train_set.shape)
print("test_set shape:",test_set.shape)
print("train_classes shape:",train_classes.shape)
print("test_classes shape:",test_classes.shape)

train_set shape: (450, 13)
test_set shape: (150, 13)
train_classes shape: (450,)
test_classes shape: (150,)


In [11]:
# DNN
# Use Keras Backend Type
train_set_d=train_set.astype(K.floatx())
test_set_d=test_set.astype(K.floatx())

In [12]:
# One Hot encode
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
integer_encoded_train_classes =  train_classes.reshape(len( train_classes), 1)
onehot_encoded_train_classes = onehot_encoder.fit_transform(integer_encoded_train_classes,1)
integer_encoded_test_classes =  test_classes.reshape(len( test_classes),1)
onehot_encoded_test_classes = onehot_encoder.fit_transform(integer_encoded_test_classes,1)

In [13]:
# Reshape Sets for Keras
train_set_d=train_set.reshape(train_set_d.shape[0],1,train_set_d.shape[1])
test_set_d=test_set.reshape(test_set_d.shape[0],1,test_set_d.shape[1])
train_classes_d_hot=onehot_encoded_train_classes.reshape(onehot_encoded_train_classes.shape[0],1,
                                                         onehot_encoded_train_classes.shape[1])
test_classes_d_hot=onehot_encoded_test_classes.reshape(onehot_encoded_test_classes.shape[0],1,
                                                       onehot_encoded_test_classes.shape[1])

In [14]:
# Check Set Shapes
print("train_set shape:",train_set_d.shape)
print("test_set shape:",train_set_d.shape)
print("train_classes shape:",train_classes_d_hot.shape)
print("test_classes shape:",test_classes_d_hot.shape)

train_set shape: (450, 1, 13)
test_set shape: (450, 1, 13)
train_classes shape: (450, 1, 6)
test_classes shape: (150, 1, 6)


In [15]:
show_web("https://keras.io/getting-started/sequential-model-guide/")

In [16]:
def create_model(fc_layers=[4],
                 activation='relu',
                 optimizer='rmsprop'):
    model = Sequential()

    for i, size in enumerate(fc_layers):
        # Input Layer - includes the input_shape
        if i == 0:
            model.add(Dense(size,
                            activation=activation,
                            input_shape=(train_set.shape[1],)))
        else:
            model.add(Dense(size,
                            activation=activation))
            
    model.add(Dense(6,activation='softmax'))
    model.compile(optimizer = optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [17]:
model = KerasClassifier(build_fn=create_model,
                        epochs=50, 
                        batch_size=5,
                        verbose=0)

In [20]:
param_grid = {'fc_layers': [[12],[12,6],[8], [4]],
              'activation':['relu','tanh'],
              'optimizer':('rmsprop','adam'),
              'epochs':[50, 100, 150],
              'batch_size':[5,10]}

In [ ]:
grid = GridSearchCV(model,
                    param_grid=param_grid,
                    return_train_score=True,
                    cv=10)

categorical_encoded_train_classes=to_categorical(integer_encoded_train_classes)
grid_results = grid.fit(train_set,onehot_encoded_train_classes, verbose=1)

print('Parameters of the best model: ')
print(grid_results.best_params_)

Epoch 1/50
405/405 [==============================] - 7s 17ms/step - loss: 2.0593 - acc: 0.1605
Epoch 2/50
405/405 [==============================] - 0s 939us/step - loss: 1.6939 - acc: 0.3383
Epoch 3/50
405/405 [==============================] - 0s 1ms/step - loss: 1.4613 - acc: 0.4222
Epoch 4/50
405/405 [==============================] - 0s 989us/step - loss: 1.2915 - acc: 0.5259
Epoch 5/50
405/405 [==============================] - 0s 984us/step - loss: 1.1571 - acc: 0.6074
Epoch 6/50
405/405 [==============================] - 0s 969us/step - loss: 1.0347 - acc: 0.6864
Epoch 7/50
405/405 [==============================] - 0s 952us/step - loss: 0.9227 - acc: 0.7556
Epoch 8/50
405/405 [==============================] - 0s 815us/step - loss: 0.8247 - acc: 0.8173
Epoch 9/50
405/405 [==============================] - 0s 969us/step - loss: 0.7381 - acc: 0.8420
Epoch 10/50
405/405 [==============================] - 0s 993us/step - loss: 0.6639 - acc: 0.8642
Epoch 11/50
405/405 [===========

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))